# Notebook: Exploración y scripts iniciales

Este notebook contiene celdas de ejemplo para:
- Crear la jerarquía de carpetas del proyecto
- Generar un manifest de videos
- Extraer frames y preparar datos
- Ejemplos de Dataset y DataLoader (esqueleto)

Sigue el README del repo para convenciones y formatos de anotación.

In [ ]:
# Section 1: Creación de estructura de carpetas (script de ejemplo)

import os

def create_project_dirs(root):
    dirs = [
        'data/raw', 'data/processed', 'data/annotations',
        'src/data', 'src/models', 'src/training', 'src/utils',
        'notebooks', 'experiments', 'models', 'logs', 'scripts', 'tests', 'docs'
    ]
    for d in dirs:
        path = os.path.join(root, d)
        os.makedirs(path, exist_ok=True)
    print('Directorios creados o existentes:')
    for d in dirs:
        print('-', d)

# Uso:
# create_project_dirs('..')  # ajustar al path deseado


In [ ]:
# Section 2: Recolección y manifest de videos (ejemplo simple)
import os
import json
import hashlib


def file_checksum(path, chunk_size=8192):
    h = hashlib.sha256()
    with open(path, 'rb') as f:
        for chunk in iter(lambda: f.read(chunk_size), b''):
            h.update(chunk)
    return h.hexdigest()


def build_manifest(videos_dir, out_json='manifest.json'):
    entries = []
    for fname in os.listdir(videos_dir):
        if not fname.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue
        p = os.path.join(videos_dir, fname)
        size = os.path.getsize(p)
        chk = file_checksum(p)
        entries.append({'file': p, 'size': size, 'sha256': chk})
    with open(out_json, 'w', encoding='utf8') as f:
        json.dump(entries, f, indent=2)
    print(f'Manifest creado: {out_json} (entries: {len(entries)})')

# Uso:
# build_manifest('data/raw', out_json='data/manifest.json')


In [ ]:
# Section 4: Extracción de frames (ffmpeg/opencv) — ejemplo
import os
import cv2


def extract_frames_cv(video_path, out_dir, fps=1):
    os.makedirs(out_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError('No se puede abrir video: %s' % video_path)
    video_fps = cap.get(cv2.CAP_PROP_FPS) or 30
    frame_interval = max(1, int(round(video_fps / fps)))
    i = 0
    saved = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if i % frame_interval == 0:
            out_path = os.path.join(out_dir, f'frame_{saved:06d}.jpg')
            cv2.imwrite(out_path, frame)
            saved += 1
        i += 1
    cap.release()
    return saved

# Uso:
# extract_frames_cv('data/raw/video.mp4', 'data/processed/video_frames', fps=1)


In [ ]:
# Section: Ejemplo de Dataset PyTorch (esqueleto)
from torch.utils.data import Dataset
import torch
import os
from PIL import Image

class VideoFramesDataset(Dataset):
    def __init__(self, frames_root, transform=None, seq_len=16):
        self.frames_root = frames_root
        # frames_root is expected to contain subfolders por clip
        self.clips = [os.path.join(frames_root, d) for d in os.listdir(frames_root) if os.path.isdir(os.path.join(frames_root, d))]
        self.transform = transform
        self.seq_len = seq_len

    def __len__(self):
        return len(self.clips)

    def __getitem__(self, idx):
        clip_folder = self.clips[idx]
        frames = sorted([os.path.join(clip_folder, f) for f in os.listdir(clip_folder) if f.endswith('.jpg')])
        # TODO: cargar secuencia de longitud seq_len, aplicar transformaciones y devolver tensor [C, T, H, W]
        imgs = [Image.open(p).convert('RGB') for p in frames[:self.seq_len]]
        if self.transform:
            imgs = [self.transform(img) for img in imgs]
        # stack to tensor [T, C, H, W] -> permute to [C, T, H, W]
        imgs = torch.stack(imgs).permute(1, 0, 2, 3)
        label = 0  # placeholder
        return imgs, label

# Uso: dataset = VideoFramesDataset('data/processed/clips', transform=...)